In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from IPython.display import display
from sklearn.feature_selection import mutual_info_regression
sns.set()

In [2]:
df = pd.read_csv('flight_data_clean.csv')
df = df.iloc[: , 1:]
df.head()

,Price,Airline,Cabin,Dept_city,Dept_date,arrival_city,stops,duration,weekday,Dept_flights_time,optimal_hours,Dep_hour,Dep_min,Arrival_hour,Arrival_min
0,3112,1,0,1,1,2,1,350,5,1,11,14,30,20,20
1,3113,2,0,1,1,2,1,255,5,2,13,16,55,21,10
2,3113,2,0,1,1,2,1,260,5,2,13,16,45,21,5
3,3113,2,0,1,1,2,1,305,5,0,4,7,55,13,0
4,3113,2,0,1,1,2,1,320,5,0,6,9,0,14,20


In [3]:
df.columns
feature = ['Airline', 'Cabin', 'Dept_city', 'Dept_date', 'arrival_city',
       'stops', 'weekday', 'Dept_flights_time',
       'Dep_hour', 'Dep_min', 'Arrival_hour', 'Arrival_min']

In [4]:
print("Correlation with price:\n")
print(df[feature].corrwith(df['Price']))
print("\n")
print("Correlation with optimal_hours:\n")
print(df[feature].corrwith(df['optimal_hours']))


Correlation with price:

Airline             -0.024121
Cabin                0.867492
Dept_city            0.053890
Dept_date           -0.096734
arrival_city         0.016511
stops                0.405831
weekday             -0.022026
Dept_flights_time    0.060233
Dep_hour             0.077847
Dep_min             -0.016910
Arrival_hour        -0.099956
Arrival_min         -0.005965
dtype: float64


Correlation with optimal_hours:

Airline              0.032674
Cabin                0.005300
Dept_city            0.036469
Dept_date            0.018700
arrival_city         0.000448
stops                0.004974
weekday             -0.061869
Dept_flights_time   -0.167815
Dep_hour            -0.134378
Dep_min              0.020950
Arrival_hour         0.049672
Arrival_min          0.006868
dtype: float64


In [15]:
cols = ['Airline', 'Dept_city', 'Dept_date', 'arrival_city',
       'stops','Cabin','Dep_hour', 'Dep_min', 'Arrival_hour', 'Arrival_min', 'Dept_flights_time', 'weekday'] 
label = ['optimal_hours','Price']

In [16]:
X = df[cols].copy()
y = df[label].copy()

## Scaling data

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
X = scaler.fit_transform(X)

#showing data
print('x \n' , X[:10])
print('y \n' , y[:10])

x 
 [[0.13953488 0.         0.65517241 0.         0.5862069  0.16666667
  0.         0.60869565 0.54545455 0.86956522 0.36363636 0.33333333
  0.83333333]
 [0.27906977 0.         0.65517241 0.         0.5862069  0.16666667
  0.         0.69565217 1.         0.91304348 0.18181818 0.66666667
  0.83333333]
 [0.27906977 0.         0.65517241 0.         0.5862069  0.16666667
  0.         0.69565217 0.81818182 0.91304348 0.09090909 0.66666667
  0.83333333]
 [0.27906977 0.         0.65517241 0.         0.5862069  0.16666667
  0.         0.30434783 1.         0.56521739 0.         0.
  0.83333333]
 [0.27906977 0.         0.65517241 0.         0.5862069  0.16666667
  0.         0.39130435 0.         0.60869565 0.36363636 0.
  0.83333333]
 [0.13953488 0.         0.65517241 0.         0.06896552 0.16666667
  0.         0.86956522 1.         0.08695652 0.18181818 0.66666667
  0.83333333]
 [0.13953488 0.         0.65517241 0.         0.06896552 0.16666667
  0.         0.65217391 0.45454545 0.8695652

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=0)
print("X_train shape is:",X_train.shape)
print("X_test shape is:",X_test.shape)
print("y_train shape is:",y_train.shape)
print("y_test shape is:",y_test.shape)

X_train shape is: (221728, 12)
X_test shape is: (109210, 12)
y_train shape is: (221728, 2)
y_test shape is: (109210, 2)


In [18]:
X_train

,Airline,Dept_city,Dept_date,arrival_city,stops,Cabin,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Dept_flights_time,weekday
323343,5,6,19,28,1,1,6,55,18,10,0,2
284996,4,20,14,4,1,2,9,15,23,35,0,4
148577,4,21,19,26,2,0,14,0,7,40,1,2
29614,6,25,6,5,1,0,9,55,16,20,0,3
295182,4,1,19,12,0,2,12,0,12,55,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
122579,26,26,16,17,1,0,7,5,13,15,0,6
304137,4,25,25,5,1,2,14,20,23,35,1,1
152315,3,4,20,2,0,0,18,0,19,40,2,3
117952,4,9,16,18,4,0,12,35,7,20,1,6


## Random forest regressor 

In [19]:
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressorModel = RandomForestRegressor(max_depth=50,random_state=0,n_jobs=-1)#n_estimators=10000,max_depth=10, n_jobs=-1,
RandomForestRegressorModel.fit(X_train, y_train)

y_pred = RandomForestRegressorModel.predict(X_test)

print('Random Forest Regressor Train Score is : ' , RandomForestRegressorModel.score(X_train, y_train))
print('Random Forest Regressor Test Score is : ' , RandomForestRegressorModel.score(X_test, y_test))
print('Random Forest Regressor No. of features are : ' , RandomForestRegressorModel.n_features_)
print('----------------------------------------------------')

print('Predicted Value for Random Forest Regressor is : ' , y_pred[:49])
print("score of model is :",RandomForestRegressorModel.score(X_test,y_test))

Random Forest Regressor Train Score is :  0.9226383736400967
Random Forest Regressor Test Score is :  0.4720209787975082
Random Forest Regressor No. of features are :  12
----------------------------------------------------
Predicted Value for Random Forest Regressor is :  [[1.29800000e+01 6.96651000e+03]
 [3.34000000e+00 1.08654800e+04]
 [1.50100000e+01 3.10550000e+04]
 [7.88000000e+00 2.29689300e+04]
 [1.02600000e+01 3.85971000e+03]
 [1.41800000e+01 3.66190000e+04]
 [1.80600000e+01 7.46435000e+03]
 [1.64860000e+01 6.29063133e+03]
 [1.75300000e+01 7.67315000e+03]
 [8.38000000e+00 7.61757000e+03]
 [1.56900000e+01 5.65214000e+03]
 [1.91500000e+01 4.85280000e+04]
 [1.49800000e+01 4.31725400e+04]
 [1.27900000e+01 9.34500000e+03]
 [1.02700000e+01 4.56807000e+03]
 [1.73000000e+01 2.12862400e+04]
 [1.02600000e+01 9.78865000e+03]
 [1.38300000e+01 8.97601000e+03]
 [1.41800000e+01 7.66054000e+03]
 [1.69500000e+01 3.60141100e+04]
 [1.41600000e+01 6.57866000e+03]
 [1.41400000e+01 7.27983000e+03]


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


score of model is : 0.4720209787975082


In [20]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import  mean_squared_error
MSEValue = mean_squared_error(y_test, y_pred, multioutput='uniform_average') # it can be raw_values
print('Mean Squared Error Value is : ', MSEValue)
print('R2 score:', r2_score(y_test, y_pred))

Mean Squared Error Value is :  1070081.370097362
R2 score: 0.4720209787975082


### XGB Regressor

In [21]:
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

estimator = XGBRegressor(learning_rate = 0.275, max_depth = 9)

# Define the model
xgbregressor = MultiOutputRegressor(estimator = estimator, n_jobs = -1).fit(X_train, y_train)
y_pred_xgb = xgbregressor.predict(X_test)

print('xgbregressor Train Score is : ' , xgbregressor.score(X_train, y_train))
print('xgbregressor Test Score is : ' , xgbregressor.score(X_test, y_test))

xgbregressor Train Score is :  0.9742334593928914
xgbregressor Test Score is :  0.9622522373447555


In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(xgbregressor, X_test, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))
print('R2 score:', r2_score(y_test, y_pred_xgb))

Mean MAE: 407.037 (8.702)
R2 score: 0.9622522373447555


In [27]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(xgbregressor, open(filename, 'wb'))

In [29]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9622522373447555


In [30]:
#saving as pickle
pickle.dump(xgbregressor, open('xgbregressor.pkl', 'wb'))

In [31]:
pickled_model = pickle.load(open('xgbregressor.pkl', 'rb'))
pickled_model.predict(X_test)

array([[1.7858629e+01, 7.4522827e+03],
       [4.5090932e-01, 1.1687565e+04],
       [1.2836351e+01, 3.1006904e+04],
       ...,
       [2.7238882e+00, 1.0383475e+04],
       [1.8901899e+01, 9.9045293e+03],
       [3.1126797e+00, 6.7493896e+03]], dtype=float32)